In [1]:
import json

import pandas as pd

from PortfolioTracker.globalpath import GlobalPath

# Instantiate GlobalPath
global_path = GlobalPath("PortfolioTracker")
# GLOBAL PATH
profitloss_gold_file_path = global_path.joinpath(
    "DATA/GOLD/ProfitLoss/ProfitLoss_data.csv"
)

print(f"Read GOLD Layer ProfitLoss data from: {profitloss_gold_file_path}")
## PRESENTATION LAYER
df = pd.read_csv(profitloss_gold_file_path)

# Assuming your DataFrame is named df
df["stock_name"] = df.apply(
    lambda row: (
        row["symbol"] if row["symbol"] == "NIFTY" else row["scrip_name"]
    ),
    axis=1,
)

# Ensure the columns are datetime64 dtype
df["open_datetime"] = pd.to_datetime(df["open_datetime"])
df["close_datetime"] = pd.to_datetime(df["close_datetime"])

# Calculate the difference in days
df["days"] = (df["close_datetime"] - df["open_datetime"]).dt.days

Read GOLD Layer ProfitLoss data from: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\GOLD\ProfitLoss\ProfitLoss_data.csv


In [2]:
# Grouping by 'name' and 'type'
result = []
grouped = df.groupby(["segment", "exchange", "symbol", "stock_name"])

for (segment, exchange, symbol, stock_name), group in grouped:
    group_dict = {
        "segment": segment,
        "exchange": exchange,
        "symbol": symbol,
        "stock_name": stock_name,
        "days": (
            group["close_datetime"].max() - group["open_datetime"].min()
        ).days,
        "quantity": group["quantity"].sum(),
        "avg_price": round(
            group["open_amount"].sum() / group["quantity"].sum(), 2
        ),
        "sell_price": round(
            group["close_amount"].sum() / group["quantity"].sum(), 2
        ),
        "pnl": group["pnl_amount"].sum(),
        "history": group[
            [
                "scrip_name",
                "position",
                "quantity",
                "days",
                "open_datetime",
                "open_price",
                "open_amount",
                "close_datetime",
                "close_price",
                "close_amount",
                "pnl_amount",
                "pnl_percentage",
            ]
        ].to_dict(orient="records"),
    }
    result.append(group_dict)

output = {
    "invested": round(df["open_amount"].sum(), 2),
    "sold": round(df["close_amount"].sum(), 2),
    "pnl": round(df["pnl_amount"].sum(), 2),
    "data": result,
}

# Writing the result to a JSON file
output_path = global_path.joinpath("DATA/PRESENTATION/API/ProfitLoss_data.json")
with open(output_path, "w") as json_file:
    json.dump(output, json_file, indent=4, default=str)

print(f"Data written to {output_path.resolve()}")

Data written to C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\PRESENTATION\API\ProfitLoss_data.json
